# DeepLabCut 2.2+ Toolbox - COLAB
![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1628180434489-T0RIWEJJU0FJVOT6FNVD/maDLC.png?format=800w)

https://github.com/DeepLabCut/DeepLabCut

This notebook illustrates how to, for multi-animal projects, use the cloud-based GPU to:
- create a multi-animal training set
- train a network
- evaluate a network
- analyze novel videos
- assemble animals and tracklets
- create quality check plots!

###This notebook assumes you already have a DLC project folder with labeled data and you uploaded it to your own Google Drive.

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the docs on GitHub. We also recommend checking out our preprint, which covers the science of maDLC

**Lauer et al 2021:** https://www.biorxiv.org/content/10.1101/2021.04.30.442096v1




## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"


In [1]:
#(this will take a few minutes to install all the dependences!)
!pip install deeplabcut
%reload_ext numpy
%reload_ext scipy
%reload_ext matplotlib
%reload_ext mpl_toolkits

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 591 kB 4.1 MB/s 
     |████████████████████████████████| 177 kB 77.1 MB/s 
     |████████████████████████████████| 9.8 MB 31.9 MB/s 
     |████████████████████████████████| 948 kB 67.4 MB/s 
     |████████████████████████████████| 11.2 MB 78.8 MB/s 
     |████████████████████████████████| 296 kB 73.4 MB/s 
     |████████████████████████████████| 109 kB 88.7 MB/s 
     |████████████████████████████████| 3.3 MB 48.1 MB/s 
     |████████████████████████████████| 352 kB 78.0 MB/s 
     |████████████████████████████████| 944 kB 67.2 MB/s 
     |████████████████████████████████| 34.5 MB 93.9 MB/s 
     |████████████████████████████████| 546 kB 92.1 MB/s 
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110474 sha256=aedb13e467f9bc9fff734dc4bee18337fadd3b952eee3f5cbcf6e6eda3bc3e7f
  Stored in directory: /root/.cache/pip/wheels/ce

In [1]:
#a few colab specific things needed:
!pip install --upgrade scikit-image
!pip3 install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.5 MB 3.7 MB/s 
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.18.3
    Uninstalling scikit-image-0.18.3:
      Successfully uninstalled scikit-image-0.18.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.4.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 256 kB 4.0 MB/s 


After the package is installed, please click "restart runtime" if it appears for DLC changes to take effect in your COLAB environment. You will see this button at the output of the cells above ^.

In [2]:
import deeplabcut
import pickle5 as pickle

Loading DLC 2.2.1.1...


/usr/local/lib/python3.7/dist-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


## Link your Google Drive (with your labeled data):

- This code assumes you locally installed DeepLabCut, created a project, extracted and labeled frames. Be sure to "check Labels" to confirm you are happy with your data. As, these frames are the only thing that is used to train your network. 💪 You can find all the docs to do this here: https://deeplabcut.github.io/DeepLabCut

- Next, place your DLC project folder into you Google Drive- i.e., copy the folder named "Project-YourName-TheDate" into Google Drive.

- Then, click run on the cell below to link this notebook to your Google Drive:

In [3]:
#Now, let's link to your Google Drive. Run this cell and follow the authorization instructions:
#(We recommend putting a copy of the github repo in your google drive if you are using the demo "examples")

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Next, edit the few items below, and click run:


In [4]:
# PLEASE EDIT THIS:
ProjectFolderName = '/content/drive/MyDrive/Deeplabcut/Silkworms-MMasaki-2022-07-20'
VideoType = 'mp4' #, mp4, MOV, or avi, whatever you uploaded!


# No need to edit this, we are going to assume you put videos you want to analyze in the "videos" folder, but if this is NOT true, edit below:
videofile_path = ['/content/drive/MyDrive/Deeplabcut/Silkworms-MMasaki-2022-07-20/videos'] #Enter the list of videos or folder to analyze.
videofile_path

#No need to edit this, as you set it when you passed the ProjectFolderName (above): 
path_config_file = '/content/drive/MyDrive/Deeplabcut/Silkworms-MMasaki-2022-07-20/config.yaml'
path_config_file
#This creates a path variable that links to your google drive project

'/content/drive/MyDrive/Deeplabcut/Silkworms-MMasaki-2022-07-20/config.yaml'

## Create a multi-animal training dataset:

- more info: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html#create-training-dataset

- please check the text below, edit if needed, and then click run (this can take some time):

In [ ]:
#OPTIONAL LEARNING: did you know you can check what each function does by running with a ?
deeplabcut.create_multianimaltraining_dataset?

In [5]:
# ATTENTION:
#which shuffle do you want to create and train?
shuffle = 1 #edit if needed; 1 is the default.

#if you labeled on Windows, please set the windows2linux=True:
# deeplabcut.create_multianimaltraining_dataset(path_config_file, Shuffles=[shuffle], net_type="dlcrnet_ms5",windows2linux=False)

## Start training:
This function trains the network for a specific shuffle of the training dataset. 
 - more info: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html#train-the-network

In [ ]:
#let's also change the display and save_iters just in case Colab takes away the GPU... 
#Typically, you want to train to 50,000 - 200K iterations.
#more info and there are more things you can set: https://github.com/DeepLabCut/DeepLabCut/blob/master/docs/functionDetails.md#g-train-the-network

deeplabcut.train_network(path_config_file, shuffle=shuffle, displayiters=100,saveiters=1000, maxiters=100000, allow_growth=True)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 50K iterations). 
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

**When you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## Start evaluating: 

 - First, we evaluate the pose estimation performance.

- This function evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images) and stores the results as .5 and .csv file in a subdirectory under **evaluation-results**

- If the scoremaps do not look accurate, don't proceed to tracklet assembly; please consider (1) adding more data, (2) adding more bodyparts!

- more info: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html#evaluate-the-trained-network

Here is an example of what you'd aim to see before proceeding:

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1590535809087-X655WY9W1MW1MY1I7DHE/ke17ZwdGBToddI8pDm48kBoswZhKnUtAF7-bTXgw67EUqsxRUqqbr1mOJYKfIPR7LoDQ9mXPOjoJoqy81S2I8N_N4V1vUb5AoIIIbLZhVYxCRW4BPu10St3TBAUQYVKc5tTP1cnANTUwNNPnYFjIp6XbP9N1GxIgAkxvBVqt0UvLpPHYwvNQTwHg8f_Zu8ZF/evaluation.png?format=1000w)



In [ ]:
#let's evaluate first:
deeplabcut.evaluate_network(path_config_file,Shuffles=[shuffle], plotting=True)
#plot a few scoremaps:
deeplabcut.extract_save_all_maps(path_config_file, shuffle=shuffle, Indices=[0])

IF these images, numbers, and maps do not look good, do not proceed. You should increase the diversity and number of frames you label, and re-create a training dataset and re-train! 

## Start Analyzing videos: 
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in a pickle file in the same directory where the video resides. 

In [6]:
print("Start Analyzing my video(s)!")
#EDIT OPTION: which video(s) do you want to analyze? You can pass a path or a folder:
# currently, if you run "as is" it assumes you have a video in the DLC project video folder!

deeplabcut.analyze_videos(path_config_file,videofile_path, shuffle=shuffle, videotype=VideoType,save_as_csv=True,identity_only='control')

Start Analyzing my video(s)!
Using snapshot-9000 for model /content/drive/MyDrive/Deeplabcut/Silkworms-MMasaki-2022-07-20/dlc-models/iteration-0/SilkwormsJul20-trainset95shuffle1


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tf_slim/layers/layers.py:684: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  outputs = layer.apply(inputs, training=is_training)


Activating extracting of PAFs
Analyzing all the videos in the directory...
Starting to analyze %  /content/drive/MyDrive/Deeplabcut/Silkworms-MMasaki-2022-07-20/videos/June5.mp4
Video already analyzed! /content/drive/MyDrive/Deeplabcut/Silkworms-MMasaki-2022-07-20/videos/June5DLC_dlcrnetms5_SilkwormsJul20shuffle1_9000.h5
Using snapshot-9000 for model /content/drive/MyDrive/Deeplabcut/Silkworms-MMasaki-2022-07-20/dlc-models/iteration-0/SilkwormsJul20-trainset95shuffle1
Processing...  /content/drive/MyDrive/Deeplabcut/Silkworms-MMasaki-2022-07-20/videos/June5.mp4
Tracklets already computed /content/drive/MyDrive/Deeplabcut/Silkworms-MMasaki-2022-07-20/videos/June5DLC_dlcrnetms5_SilkwormsJul20shuffle1_9000_el.pickle
Set overwrite = True to overwrite.
The tracklets were created (i.e., under the hood deeplabcut.convert_detections2tracklets was run). Now you can 'refine_tracklets' in the GUI, or run 'deeplabcut.stitch_tracklets'.
Processing...  /content/drive/MyDrive/Deeplabcut/Silkworms-MMa

100%|██████████| 406/406 [00:04<00:00, 94.75it/s] 
/usr/local/lib/python3.7/dist-packages/deeplabcut/refine_training_dataset/stitch.py:676: UserWarning: No optimal solution found. Employing black magic...
  warnings.warn("No optimal solution found. Employing black magic...")


The videos are analyzed. Time to assemble animals and track 'em... 
 Call 'create_video_with_all_detections' to check multi-animal detection quality before tracking.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


'DLC_dlcrnetms5_SilkwormsJul20shuffle1_9000'

Optional: Now you have the option to check the raw dections before animals are assembled. To do so, pass a video path:

In [ ]:
##### PROTIP: #####
## look at the output video; if the pose estimation (i.e. key points)
## don't look good, don't proceed with tracking - add more data to your training set and re-train!

#EDIT: let's check a specific video (PLEASE EDIT VIDEO PATH):
Specific_videofile = '/content/drive/MyDrive/Deeplabcut/Kaico-MasukoMasaki-2022-07-03/videos/Kaico.mp4'

#don't edit:
deeplabcut.create_video_with_all_detections(path_config_file, [Specific_videofile], shuffle=shuffle)

Creating labeled video for  Kaico


error: ignored

If the resutling video (ends in full.mp4) is not good, we highly recommend adding more data and training again. See here: https://deeplabcut.github.io/DeepLabCut/docs/maDLC_UserGuide.html#decision-break-point

# Next, we will assemble animals using our data-driven optimal graph method:

- Here, we will find the optimal graph, which matches the "data-driven" method from our paper (Figure adapted from Lauer et al. 2021):

![alt text](https://images.squarespace-cdn.com/content/v1/57f6d51c9f74566f55ecf271/1626266017809-XO6NX84QB4FBAZGOTCEY/fig3.jpg?format=400w)


- note, you can set the number of animals you expect to see, so check, edit, then click run:

In [ ]:
#Check and edit:
numAnimals = 4 #how many animals do you expect to find?
tracktype= 'ellipse' #box, skeleton, ellipse:
#-- ellipse is recommended, unless you have a single-point ma project, then use BOX!

#Optional: 
#imagine you tracked a point that is not useful for assembly, 
#like a tail tip that is far from the body, consider dropping it for this step (it's still used later)!
#To drop it, uncomment the next line TWO lines and add your parts(s):

#bodypart= 'Tail_end'
#deeplabcut.convert_detections2tracklets(path_config_file, videofile_path, videotype=VideoType, shuffle=shuffle, overwrite=True, ignore_bodyparts=[bodypart])

#OR don't drop, just click RUN:
deeplabcut.convert_detections2tracklets(path_config_file, videofile_path, videotype=VideoType, 
                                        shuffle=shuffle, overwrite=True)

deeplabcut.stitch_tracklets(path_config_file, videofile_path, shuffle=shuffle, track_method=tracktype, n_tracks=numAnimals)

Using snapshot-9000 for model /content/drive/MyDrive/Deeplabcut/Silkworms-MMasaki-2022-07-20/dlc-models/iteration-0/SilkwormsJul20-trainset95shuffle1
Analyzing all the videos in the directory...
Processing...  /content/drive/MyDrive/Deeplabcut/Silkworms-MMasaki-2022-07-20/videos/June5.mp4
Analyzing /content/drive/MyDrive/Deeplabcut/Silkworms-MMasaki-2022-07-20/videos/June5DLC_dlcrnetms5_SilkwormsJul20shuffle1_9000.h5


100%|██████████| 8997/8997 [00:10<00:00, 848.21it/s]
8997it [00:22, 396.32it/s]


The tracklets were created (i.e., under the hood deeplabcut.convert_detections2tracklets was run). Now you can 'refine_tracklets' in the GUI, or run 'deeplabcut.stitch_tracklets'.
Analyzing all the videos in the directory...
Processing...  /content/drive/MyDrive/Deeplabcut/Silkworms-MMasaki-2022-07-20/videos/June5.mp4


100%|██████████| 406/406 [00:06<00:00, 66.52it/s]


Now let's filter the data to remove any small jitter:

In [ ]:
deeplabcut.filterpredictions(path_config_file, 
                                 videofile_path, 
                                 shuffle=shuffle,
                                 videotype=VideoType, 
                                 track_method = tracktype)

Analyzing all the videos in the directory...
Filtering with median model /content/drive/MyDrive/Deeplabcut/Silkworms-MMasaki-2022-07-20/videos/June5.mp4
Saving filtered csv poses!


## Create plots of your trajectories:

In [ ]:
deeplabcut.plot_trajectories(path_config_file, videofile_path, videotype=VideoType, shuffle=shuffle, track_method=tracktype)

Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4. 

## Create labeled video:
This function is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
deeplabcut.create_labeled_video(path_config_file,
                                videofile_path, 
                                shuffle=shuffle, 
                                color_by="individual",
                                videotype=VideoType, 
                                save_frames=False,
                                filtered=True,
                                keypoints_only=True)

In [ ]:
deeplabcut.refine_tracklets(path_config_file, pickle_or_h5_file, videofile_path, max_gap=0, min_swap_len=2, min_tracklet_len=2, trail_len=50)